# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 13 2023 12:55PM,255019,10,PRF-46683,Bio-PRF,Released
1,Jan 13 2023 12:55PM,255019,10,PRF-46689,Bio-PRF,Released
2,Jan 13 2023 12:55PM,255019,10,PRF-46693,Bio-PRF,Released
3,Jan 13 2023 12:32PM,255015,18,INNOVA-9023102,Innova Softgel LLC,Released
4,Jan 13 2023 12:18PM,255011,10,0086298208,ISDIN Corporation,Released
5,Jan 13 2023 12:18PM,255011,10,0086298209,ISDIN Corporation,Released
6,Jan 13 2023 12:18PM,255011,10,0086298212,ISDIN Corporation,Released
7,Jan 13 2023 12:18PM,255011,10,0086298214,ISDIN Corporation,Released
8,Jan 13 2023 12:18PM,255011,10,0086298213,ISDIN Corporation,Released
9,Jan 13 2023 12:18PM,255011,10,0086298211,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,255009,Executing,2
20,255009,Released,3
21,255011,Released,13
22,255015,Released,1
23,255019,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255004,NaN,4.0,1.0
255009,NaN,2.0,3.0
255011,NaN,NaN,13.0
255015,NaN,NaN,1.0
255019,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254949,0.0,0.0,4.0
254961,0.0,1.0,0.0
254976,6.0,5.0,37.0
254980,5.0,3.0,4.0
254992,0.0,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254949,0,0,4
254961,0,1,0
254976,6,5,37
254980,5,3,4
254992,0,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254949,0,0,4
1,254961,0,1,0
2,254976,6,5,37
3,254980,5,3,4
4,254992,0,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254949,,,4
1,254961,,1,
2,254976,6,5,37
3,254980,5,3,4
4,254992,,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 13 2023 12:55PM,255019,10,Bio-PRF
3,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC
4,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation
17,Jan 13 2023 12:06PM,255009,10,ISDIN Corporation
22,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation
27,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd"
28,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC"
33,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC"
34,Jan 13 2023 9:45AM,254997,10,Bio-PRF
55,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 13 2023 12:55PM,255019,10,Bio-PRF,,,3
1,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,,,1
2,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation,,,13
3,Jan 13 2023 12:06PM,255009,10,ISDIN Corporation,,2,3
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,,4,1
5,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",,1,
6,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",,,5
7,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",,,1
8,Jan 13 2023 9:45AM,254997,10,Bio-PRF,,10,11
9,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc.",5,57,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 12:55PM,255019,10,Bio-PRF,3,,
1,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,1,,
2,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation,13,,
3,Jan 13 2023 12:06PM,255009,10,ISDIN Corporation,3,2,
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,1,4,
5,Jan 13 2023 10:39AM,255002,19,"AdvaGen Pharma, Ltd",,1,
6,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",5,,
7,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",1,,
8,Jan 13 2023 9:45AM,254997,10,Bio-PRF,11,10,
9,Jan 13 2023 9:44AM,254994,15,"Person & Covey, Inc.",7,57,5


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 12:55PM,255019,10,Bio-PRF,3,,
1,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,1,,
2,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation,13,,
3,Jan 13 2023 12:06PM,255009,10,ISDIN Corporation,3,2,
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,1,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 12:55PM,255019,10,Bio-PRF,3.0,NaN,NaN
1,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,1.0,NaN,NaN
2,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation,13.0,NaN,NaN
3,Jan 13 2023 12:06PM,255009,10,ISDIN Corporation,3.0,2.0,NaN
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,1.0,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 12:55PM,255019,10,Bio-PRF,3.0,0.0,0.0
1,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,1.0,0.0,0.0
2,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation,13.0,0.0,0.0
3,Jan 13 2023 12:06PM,255009,10,ISDIN Corporation,3.0,2.0,0.0
4,Jan 13 2023 10:54AM,255004,10,ISDIN Corporation,1.0,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1784969,39.0,19.0,5.0
12,254961,0.0,1.0,0.0
15,1019977,21.0,57.0,5.0
18,255015,1.0,0.0,0.0
19,509978,37.0,6.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1784969,39.0,19.0,5.0
1,12,254961,0.0,1.0,0.0
2,15,1019977,21.0,57.0,5.0
3,18,255015,1.0,0.0,0.0
4,19,509978,37.0,6.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,39.0,19.0,5.0
1,12,0.0,1.0,0.0
2,15,21.0,57.0,5.0
3,18,1.0,0.0,0.0
4,19,37.0,6.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,12,Released,0.0
2,15,Released,21.0
3,18,Released,1.0
4,19,Released,37.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19
Status,,,,,
Completed,5.0,0.0,5.0,0.0,6.0
Executing,19.0,1.0,57.0,0.0,6.0
Released,39.0,0.0,21.0,1.0,37.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19
0,Completed,5.0,0.0,5.0,0.0,6.0
1,Executing,19.0,1.0,57.0,0.0,6.0
2,Released,39.0,0.0,21.0,1.0,37.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19
0,Completed,5.0,0.0,5.0,0.0,6.0
1,Executing,19.0,1.0,57.0,0.0,6.0
2,Released,39.0,0.0,21.0,1.0,37.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()